In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta

In [174]:
%run ../interesting_options.py
# %run -i my_3dbar_plotter.py

In [3]:
# %reset_selective -f plotter
%who_ls

['add_z_score',
 'calculate_horizon_date',
 'date',
 'datetime',
 'datetime_valid',
 'filter_by_date',
 'filter_by_z_score',
 'interesting_options',
 'max_interest_by',
 'np',
 'parse_contractSymbol',
 'pd',
 'plt',
 'timedelta']

In [175]:
# load calls option chains
calls1, calls2, calls3, calls4 = (
#     pd.read_csv('D:\\code\\test\\csv\\pltr\\calls1.csv', index_col=0),
#     pd.read_csv('D:\\code\\test\\csv\\pltr\\calls2.csv', index_col=0),
#     pd.read_csv('D:\\code\\test\\csv\\pltr\\calls3.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls1.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls2.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls3.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls4.csv', index_col=0),
)
print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)
print('calls4', calls4.shape)

# filter for interesting options
calls1, calls2, calls3, calls4 = (
    interesting_options(calls1, time_horizon='3mo', z_score_let=2, rmold=True),
    interesting_options(calls2, time_horizon='3mo', z_score_let=2, rmold=True),
    interesting_options(calls3, time_horizon='3mo', z_score_let=2, rmold=True),
    interesting_options(calls4, time_horizon='3mo', z_score_let=2, rmold=True),
)
# calls1.loc[calls1['exp'] < date.today()]
print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)
print('calls4', calls4.shape)

calls1 (436, 20)
calls2 (387, 20)
calls3 (391, 20)
calls4 (389, 20)
calls1 (12, 23)
calls2 (11, 23)
calls3 (13, 23)
calls4 (10, 23)


In [7]:
# my_3dbar_plotter accepts a misdf
def o_chains_to_3d_df(o_chains, 
                      x='date_strike', 
                      y='columns', 
                      z='openInterest', 
                      colsasdays=True
                     ):
    """
    Takes a number of option chains (each == one timeperiod of observation, default = 1 day).
    Returns a df where: 
        x = index (expiration date for MIS, or contract symbol) - x-axis
        y = column (observation time period, day1, day2 etc) - y-axis
        z = whatever we are looking at (open interest for MIS) - z-axis
        
    By default we count number of o_chains and create colums for each o_chain: colsasdays=True
    """
    
    # TODO - else y is dodgy
    columns = np.arange(0, len(o_chains), 1) if colsasdays is True else y 
    
    # I need to combine all o_chains indexes to make enough room in 3D bar plot
    # slice each chain
    def slice_chains(chains, x, y, z):
        out = []
        for i, chain in enumerate(chains):
         #   print(chain['exp']) # omg fuck this!!! introducing duplicates!
            new_chain = pd.DataFrame(chain[z].values, columns=[i], index=chain['date_strike'])
            out.append(new_chain)
        return out
   
    sliced = slice_chains(o_chains, x, y, z)
    out_df = pd.concat(sliced, axis=1)
    
    return out_df
    

In [179]:
# now plot a 3D bar chart
# x = expirations
# y = days of observation
# z = open_interest
mis_df = o_chains_to_3d_df([calls4, calls3, calls2, calls1])
# print(mis_df.shape)
# mis_df.to_csv('~/Desktop/code/mamacita/algo2/csv/pltr/mis_df2.csv')
# mis_df.to_csv('D:\\code\\test\\csv\\pltr\\mis_df1.csv') # damn df_combiner botches up

In [178]:
def df_to_3Dbars(misdf):
    # MIS DataFrame is
    # rows == days of observations
    # cols = expirations
    # values = MIS
    arr = []

    for idx in misdf.index:
        for col in misdf.columns:
            bar = [idx, col, misdf.loc[idx, col]]
            arr.append(bar)

    x = []
    y = []
    z = []

    for bar in arr:
        idx, col, strike = bar
        x.append(idx)
        y.append(col)
        z.append(strike)

    return x, y, z


def plotter(x, y, z, 
            width=1, 
            depth=1, 
            ax=False, 
            figsize=False, 
            labels=['x', 'y', 'z'], 
            xticks=False, 
            xticklabels=False
           ):

    # defaults
    figsize = (10, 10) if figsize is False else figsize

    if ax is False:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection='3d')

    bottom = np.zeros_like(z)
    hight = z

#     ax.set_xlabel(labels[0])
    ax.set_ylabel(labels[1])
    ax.set_zlabel(labels[2])
    
    # set xticks and xticks labels
    # https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.axes.Axes.set_xticks.html
    # https://stackoverflow.com/questions/14852821/aligning-rotated-xticklabels-with-their-respective-xticks
    if xticks is not False:
        print('len xticks:', len(xticks))
        ax.set_xticks(xticks, 
                      minor=False
                     )
    else:
        ax.set_xticks(range(0, len(x))) # or default
    
    if xticklabels is not False:
        # https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.axes.Axes.set_xticklabels.html
        # print('xticklabels:', xticklabels)
        ax.set_xticklabels(xticklabels,
                          rotation=45,
                          ha='right'
                          )
    
    out = ax.bar3d(x, y, bottom, width, depth, hight)
    return out


def my_3dbar_plotter(misdf, **kwargs):
    x, y, z = df_to_3Dbars(misdf.loc[:, misdf.columns != 'index'])
    return plotter(x, y, z, 
                   xticks=range(0,len(misdf['index'])), 
                   xticklabels=misdf['index'],
                   **kwargs
                  )

In [180]:
df.head()

,index,0,1,2,3
0,2021-01-29 30.0,25091.0,16341.0,13497.0,0
1,2021-02-05 30.0,10253.0,8616.0,7827.0,7827
2,2021-02-12 30.0,4200.0,2162.0,2073.0,0
3,2021-02-19 27.0,36912.0,35470.0,33786.0,0
4,2021-02-19 30.0,45602.0,41350.0,38639.0,0


In [181]:
# df = mis_df.reset_index(drop=True)
# remove past expirations
# df.head()
# df = df.loc[:, df['index'] > datetime.today()]
df = mis_df.reset_index()
df = df.fillna(0)
df.shape # (22, 5)


(19, 5)

In [182]:
%matplotlib qt 
print('df[index]', len(df['index']))
my_3dbar_plotter(df, 
                 depth=1, 
                 labels=['date_strike', 'days', 'open_interest']
                )

df[index] 19
len xticks: 19
